# Trees in Paris

In [ ]:
from json import load

from ipywidgets import Text, VBox
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale

from py2vega.functions.color import rgb

from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr

with open('./trees.json') as fobj:
    data = load(fobj)

# Get list of possible tree species and kinds for the ordinal color scales
species = list(set([tree['Species'] for tree in data['data'] if tree['Species'] is not None]))
kinds = list(set([tree['Kind'] for tree in data['data'] if tree['Kind'] is not None]))

# Text box for highlighting tree names
highlight_box = Text(description='Highlight: ', value='Platane')
name_background_color = Expr("'yellowgreen' if '{}' in value else 'white'".format(highlight_box.value))

def on_textbox_changed(*args, **kwargs):
    name_background_color.value = "'yellowgreen' if '{}' in value else 'white'".format(highlight_box.value)

highlight_box.observe(on_textbox_changed, 'value')

def circumference_color(value):
    color_value = 255 if value > 300 else (value / 300) * 255
    return rgb(0, color_value, color_value)

renderers = {
    'French_name': TextRenderer(
        background_color=name_background_color
    ),
    'Height_m': BarRenderer(
        bar_value=LinearScale(min=0, max=30),
        bar_color=ColorScale(min=0, max=30, scheme='viridis'),
        horizontal_alignment='center',
    ),
    'Species': TextRenderer(
        background_color=OrdinalColorScale(domain=species)
    ),
    'Kind': TextRenderer(
        background_color=OrdinalColorScale(domain=kinds)
    ),
    'Type': TextRenderer(
        missing='Unspecified'
    ),
    'Circumference_cm': BarRenderer(
        bar_value=LinearScale(min=0, max=300),
        bar_color=Expr(circumference_color),
        horizontal_alignment='center'
    ),
}

datagrid = DataGrid(data=data, base_row_size=32, base_column_size=90, renderers=renderers)
VBox((highlight_box, datagrid))

In [ ]:
def circumference_color(value):
    color_value = 255 if value > 300 else (value / 300) * 255
    return rgb(color_value, 0, 0)

renderers['Circumference_cm'].bar_color.value = circumference_color

In [ ]:
datagrid.transform([
    {'type': 'sort', 'columnIndex': 5},
    {'type': 'filter', 'operator': '=', 'columnIndex': 3, 'value': 'Platane'}
])